<a href="https://cognitiveclass.ai"><img src = "https://ibm.box.com/shared/static/9gegpsmnsoo25ikkbl4qzlvlyjbgxs5x.png" width = 400> </a>

# Project Capstone - Comparing quantity of Restaurants in Airports with more passenger traffic #

## Introduction

The Airport restaurants with more passenger traffic need to serve a diverse and multicultural population. Most people choose airports fast food restaurant because of lack of time between airplane connections but other people eat at the fast food restaurant because there are no other restaurants. The analysis of the number of restaurants per airport can indicate the percentage of fast-food restaurants and other restaurants and give new insights on the proposition.

In order to obtain the data about the restaurants near to airports with more passenger traffic in the world, we present in this document the process to obtain the dataset that is in the table of airports and how explore the dataset available freely in the wikipedia page. Futhermore, we explain how the data will be transformed and stored into a pandas dataframe. Then, we collect the longitude and latitude from each airport list using the Geocoder API.

Finally, the Capstone project will compare the number of restaurants within a 500-meter radius among the top 10 airports in the world in 2017, with more passenger traffic using the foursquare location data.

## Table of Contents

1. <a href="#item1">Data</a>

2. <a href="#item2">Methodology</a>

3. <a href="#item3">Results</a>

4. <a href="#item4">Discussion</a>

5. <a href="#item5">Conclusion</a>


Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests

## 1. Data

The dataset to explore the busiest airports by passenger traffic is the wikipedia site.

https://en.wikipedia.org/wiki/List_of_busiest_airports_by_passenger_traffic

In addition to the dataset from the wikipedia page, we get airport geolocalization using **the Foursquare location data** and the Venue Categories with type "Food": 

https://developer.foursquare.com/docs/resources/categories


In [2]:
wikipedia_link = 'https://en.wikipedia.org/wiki/List_of_busiest_airports_by_passenger_traffic'

In [3]:
airports_wikipedia_page = requests.get(wikipedia_link)

## 2. Methodology

## 3. Results

## 4. Discussion

## 5. Conclusion

### About the Author:  
 [Clayton Magalhaes]( https://www.linkedin.com/in/cvianam/) Clayton Magalhaes is a Fraud Prevention Specialist at IBM.



 <hr>
Copyright &copy; 2018 [cognitiveclass.ai](cognitiveclass.ai?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).